# Chapter 2 Overview: Elliptic Curve Cryptography (ECC)

## 📘 What This Chapter Explores
- Introduction to the mathematics behind elliptic curves, which are the foundation of Bitcoin’s public key cryptography  
- How points are defined and added on an elliptic curve over real numbers  
- The rules and special cases involved in point addition, including:  
  - Adding two distinct points  
  - Point doubling  
  - Identity element (point at infinity)  
  - When the result of addition is the point at infinity  
- How to implement elliptic curve point arithmetic in Python, including:  
  - The `Point` class with `__add__`, `__eq__`, and other magic methods  
  - Handling edge cases like point at infinity and vertical tangents

## 🎯 What You Should Understand by the End
- The definition and structure of an elliptic curve:  
  \[
  y^2 = x^3 + ax + b
  \]
- The concept of a group law and why elliptic curves form a group  
- The geometry of point addition on elliptic curves, including visual intuition  
- How to compute the sum of two points algebraically, using slope formulas  
- The importance of the point at infinity as the identity element  
- How to implement and test elliptic curve point operations in Python  
- Why this math matters: ECC is the basis for Bitcoin’s keys, signatures, and security


In [7]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
import helper

from helper import run
import ecc

from ecc import Point

Using the curve in fig 2-9

$y^2 = x^3 + 5x + 7$

defined in general form in the `ecc.py` file, we're interested in the following points:

In [8]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
# p2 = Point(-1, -2, 5, 7)

running p2 will raise the ValueError from the ecc file advising the point is not on the curve. p2 is simply written to show that it is not on the curve, no need to change

# Exercise 1

Determine which of these points are on the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7:

(2,4), (-1,-1), (18,77), (5,7)

In [9]:
# Exercise 1

# (2,4), (-1,-1), (18,77), (5,7)
# equation in python is: y**2 == x**3 + 5*x + 7

In [10]:
# Curve parameters
a = 5
b = 7

# List of points to check
points = [(2, 4), (-1, -1), (18, 77), (5, 7)]

# Check each point
for x, y in points:
    left = y**2
    right = x**3 + a*x + b
    if left == right:
        print(f"Point ({x}, {y}) is ON the curve.")
    else:
        print(f"Point ({x}, {y}) is NOT on the curve.")


Point (2, 4) is NOT on the curve.
Point (-1, -1) is ON the curve.
Point (18, 77) is ON the curve.
Point (5, 7) is NOT on the curve.


# Exercise 2

Write the `__ne__` method for `Point`.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_ne`

In [11]:
import importlib
import ecc
importlib.reload(ecc)

<module 'ecc' from '/content/programmingbitcoin/code-ch02/ecc.py'>

In [12]:
ls

answers.py      ecc.py       helper.py    __pycache__/
Chapter2.ipynb  examples.py  jupyter.txt


In [13]:
# Exercise 2

reload(ecc)
run(ecc.PointTest("test_ne"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


The purpose of definiing the method `__ne__` in this way is to avoid writing the entire definition, we're using the same logic definied with `__eq__`.

```python
def __ne__(self, other):
    return not (self == other)
```

Commutative addition: adding point A to B gives the same result as adding point B to point A. Elliptic curves use 'point addition' which adds two points that are on the curves to give a third point on also on the curve.

lines on a curve can intercept once or three times. It's also possible to intercept two times, where one of the two interceptions is tangent to the curve. The other case is when the line is vertical, and therefore must intercet at two points.

Point addition can be not easily predictable.

Point addition satisfies identity, commutativity, associativity, invertibility.

Coding point addition:
1.

`Inf` doesn't work well in python so we use `None`.

We want the following to work

```python

p1 = Point(-1, -1, 5, 7)
p2 = Point(-1, 1, 5, 7)
inf = Point(None, None, 5, 7)
print(p1 + inf)
print(inf + p2)
print(p1 + p2)

```

First change the `__init__` method to prevent checking the curve equation is satisfied when a point at inifity is provided.

Secondly, overload the addition operator (` __add__`).

**are methods contained in classes?(yes, but not sure if methods are limited to being contained in classes), `FieldElement` is a class, is `__init__` a method in the `Point` class? (yes)**

In [14]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
p2 = Point(-1, 1, 5, 7)
inf = Point(None, None, 5, 7)

print(p1 + inf)

Point(-1,-1)_5_7


Above output is p1 (itself). Adding inf to any point returns returns itself.

This uses the *Identity* property of *Point Addition*:

*I + A = A*

Where *A* is point *A* and *I* is a point at inf.  

In [15]:
print(inf + p2)

Point(-1,1)_5_7


Testing the *Identity* property again with p2 is correct.

To get the following point addition to work we have to understand what is going on. p1 and p2 have the same x coordinate but opposite y coordinate. This results in the **Case 1** (a point at infinity)

In [16]:
import importlib
import ecc
importlib.reload(ecc)

from ecc import Point

In [17]:
print(p1 + p2)

Point(infinity)


It is important to import the updated `Point` class again

# Exercise 3


Handle the case where the two points are additive inverses. That is, they have the same `x`, but a different `y`, causing a vertical line. This should return the point at infinity.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add0`

This was done in the previous exercise when we added p1 and p2, the x coordinate is the same, but the y coordinate is the inverse, which creates a vertical line.

In [18]:
# Exercise 3

reload(ecc)
run(ecc.PointTest("test_add0"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Adding two points with different x coordinates

$P_1 = (x_1, y_1)$, $P_2 = (x_2, y_2)$, $P_3 = (x_3, y_3)$

$P_1 + P_2 = P_3$

Slope, $s = (y_2 - y_1) / (x_2 - x_1)$

To find $x_3$,

$x_3 = s^2 - x_1 - x_2$

$y_3 = s(x_1 - x_3) - y_1$

$y_3$ is the reflection over the x-axis, hence the flip in sign. **Original point slope form for line equation:**

$y = s(x - x_1) + y_1$

where $P_3$ becomes the point of interest $(x,y)$.

Using the elliptic curve equation and the equation of the line thoug $P_1$ and $P_2$:

$y^2 = x^3 + ax + b$

$[s(x - x_1) + y_1)]^2 = x^3 + ax + b$

Gathering the terms, we have a polynomial equation. we know that $x_1, x_2, x_3$ are solutions to the polynomial equation:

$(x-x_1)(x-x_2)(x-x_3)= 0$

We can expand this and compare coefficients of the two polynonimals, using Vieta's formula (if roots are the same, so are coefficients).

Investigating the coefficient for $x^2$ is most most simple to compare.

...

$-s^2 = -(x_1 + x_2 + x_3)$

$x_3 = s^2 -x_1 - x_2$

again, point slope form for line equation:

$y = s(x - x_1 ) +y_1 $

**Since elliptic curve group law defines point addition as reflection over x-axis:**

$y = s(x - x_1 ) +y_1 \rightarrow y\prime = -s(x - x_1 ) -y_1$

$y\prime = y_3 = s(x_1 - x3) +y_1$

Now we can solve $x_3, y_3$ in terms of $s, x_1, y_1, x_2, y_2$

# Exercise 4


For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (2,5) + (-1,-1)?

In [19]:
# Exercise 4

from ecc import Point

a = 5
b = 7
x1, y1 = 2, 5
x2, y2 = -1, -1

# (x1,y1) + (x2,y2)

$s = (y_2 - y_1) / (x_2 - x_1)$

$x_3 = s^2 - x_1 - x_2$

$y_3 = s(x_1 - x_3) - y_1$

In [27]:
s = (y2 - y1) / (x2 - x1)
x3 = s**2 - x1 - x2
y3 = s * (x1 - x3) - y1

print(f"P₁ + P₂ = P₃ = ({x3}, {y3})")

P₁ + P₂ = P₃ = (3.0, -7.0)


Now that we have completed point addition when the x coordinates are different, we can update our `__add__` method.

# Exercise 5

The following exercise was completed in a previous exercise.

Write the `__add__` method where \\(x_{1}\\)≠\\(x_{2}\\)

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add1`

In [28]:
# Exercise 5

reload(ecc)
run(ecc.PointTest("test_add1"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


# Exercise 6


Point addition when $P_1= P_2$.

$P_1 = -P_2$

or

$P_1 + P_2 = I$

and

$s=\frac{dx}{dy}$

The derivative of the elliptic curve equation is

$(2y)dy = (3x^2 + a)dx$

**$s = (3x^2 + a)/2y$**

For $x_3,y_3$,

$x_3 = s^2 - x_1 - x_2= s^2 - x_1 - x_1$

**$x_3 = s^2 - 2x_1$**

and

**$y_3 = s(x_1 - x_3) - y_1$**


For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (-1,-1) + (-1,-1)?

In [29]:
# Exercise 6

from ecc import Point

a = 5
b = 7
x1, y1 = -1, -1
# (-1,-1) + (-1,-1)

In [30]:
s = (3*x1**2+ a)/2*y1
x3 = s**2 - 2*x1
y3 = s*(x1 - x3) - y1

print(f"P₁ + P₂ = P₃ = ({x3}, {y3})")

P₁ + P₂ = P₃ = (18.0, 77.0)


# Exercise 7

The following exercise was completed in a previous exercise.

Write the `__add__` method when \\(P_{1}\\)=\\(P_{2}\\).

**Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add2`**

In [31]:
# Exercise 7

reload(ecc)
run(ecc.PointTest("test_add2"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
